In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%load_ext tensorboard
import tensorflow as tf
import datetime
log_dir = "logs/my_board/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# 텐서보드 콜백 정의 하기
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
import os
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Model, model_from_json
from tensorflow.keras.layers import Input, GRU, Conv2D, Dropout, Flatten, Dense, Reshape, Concatenate, Add
from tensorflow.keras.optimizers import SGD, RMSprop, Adam

from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard

#######################################################################################################################
#                                      Start Skip RNN specific layers subsclass                                       #
#                                                                                                                     #
# The SkipRNN layer is implemented as follows:                                                                        #
# - Pre Transformation layer that takes a 'window' size of data and apply couple of reshapes and axis transformation  #
#   in order to simulate the skip RNN that is described in the paper                                                  #
# - Apply a normal GRU RNN on the transformed data. This way not the adjacent data points are connected but rather    #
#   data points that are 'skip' away.                                                                                 #
# - Post Transformation layer that brings back dimensions to its original shape as PreTrans has changed all           #
#   dimensions including Batch Size                                                                                   #
#######################################################################################################################

class PreSkipTrans(tf.keras.layers.Layer):
    def __init__(self, pt, skip, **kwargs):
        #
        # pt:   Number of different RNN cells = (window / skip)
        # skip: Number of points to skip
        #
        self.pt   = pt
        self.skip = skip
        super(PreSkipTrans, self).__init__(**kwargs)
        
    def build(self, input_shape):
        super(PreSkipTrans, self).build(input_shape)
    
    def call(self, inputs):
        # Get input tensors; in this case it's just one tensor
        x = inputs

        # Get the batchsize which is tf.shape(x)[0] since inputs is either X or C which has the same
        # batchsize as the input to the model
        batchsize = tf.shape(x)[0]

        # Get the shape of the input data
        input_shape = K.int_shape(x)
        
        # Create output data by taking a 'window' size from the end of input (:-self.pt * self.skip)
        output = x[:,-self.pt * self.skip:,:]
        
        # Reshape the output tensor by:
        # - Changing first dimension (batchsize) from None to the current batchsize
        # - Splitting second dimension into 2 dimensions
        output = tf.reshape(output, [batchsize, self.pt, self.skip, input_shape[2]])
        
        # Permutate axis 1 and axis 2
        output = tf.transpose(output, perm=[0, 2, 1, 3])
        
        # Reshape by merging axis 0 and 1 now hence changing the batch size
        # to be equal to current batchsize * skip.
        # This way the time dimension will only contain 'pt' entries which are
        # just values that were originally 'skip' apart from each other => hence skip RNN ready
        output = tf.reshape(output, [batchsize * self.skip, self.pt, input_shape[2]])
        
        # Adjust the output shape by setting back the batch size dimension to None
        output_shape = tf.TensorShape([None]).concatenate(output.get_shape()[1:])
        
        return output
    
    def compute_output_shape(self, input_shape):
        # Since the batch size is None and dimension on axis=2 has not changed,
        # all we need to do is set shape[1] = pt in order to compute the output shape
        shape = tf.TensorShape(input_shape).as_list()
        shape[1] = self.pt
        
        return tf.TensorShape(shape)


    def get_config(self):
        config = {'pt': self.pt, 'skip': self.skip}
        base_config = super(PreSkipTrans, self).get_config()
        
        return dict(list(base_config.items()) + list(config.items()))


    @classmethod
    def from_config(cls, config):
        return cls(**config)


class PostSkipTrans(tf.keras.layers.Layer):
    def __init__(self, skip, **kwargs):
        #
        # skip: Number of points to skip =24
        #
        self.skip = skip
        super(PostSkipTrans, self).__init__(**kwargs)
    
    def build(self, input_shape):
        super(PostSkipTrans, self).build(input_shape)
    
    def call(self, inputs):
        # Get input tensors
	# - First one is the output of the SkipRNN layer which we will operate on
	# - The second is the oiriginal model input tensor which we will use to get
	#   the original batchsize
        x, original_model_input = inputs

        # Get the batchsize which is tf.shape(original_model_input)[0]
        batchsize = tf.shape(original_model_input)[0]

        # Get the shape of the input data
        input_shape = K.int_shape(x)

        # Split the batch size into the original batch size before PreTrans and 'Skip'
        output = tf.reshape(x, [batchsize, self.skip, input_shape[1]])
        
        # Merge the 'skip' with axis=1
        output = tf.reshape(output, [batchsize, self.skip * input_shape[1]])
        
        # Adjust the output shape by setting back the batch size dimension to None
        output_shape = tf.TensorShape([None]).concatenate(output.get_shape()[1:])

        return output
    
    def compute_output_shape(self, input_shape):
        # Adjust shape[1] to be equal to shape[1] * skip in order for the 
        # shape to reflect the transformation that was done
        shape = tf.TensorShape(input_shape).as_list()
        shape[1] = self.skip * shape[1]
        
        return tf.TransformShape(shape)


    def get_config(self):
        config = {'skip': self.skip}
        base_config = super(PostSkipTrans, self).get_config()
        
        return dict(list(base_config.items()) + list(config.items()))


    @classmethod
    def from_config(cls, config):
        return cls(**config)


#######################################################################################################################
#                                      End Skip RNN specific layers subsclass                                         #
#######################################################################################################################


#######################################################################################################################
#                                      Start AR specific layers subsclass                                             #
#                                                                                                                     #
# The AR layer is implemented as follows:                                                                             #
# - Pre Transformation layer that takes a 'highway' size of data and apply a reshape and axis transformation          #
# - Flatten the output and pass it through a Dense layer with one output                                              #
# - Post Transformation layer that bring back dimensions to its original shape                                        #                                                     #
#######################################################################################################################

class PreARTrans(tf.keras.layers.Layer):
    def __init__(self, hw, **kwargs):
        #
        # hw: Highway = Number of timeseries values to consider for the linear layer (AR layer)
        #
        self.hw = hw
        super(PreARTrans, self).__init__(**kwargs)
    
    def build(self, input_shape):
        super(PreARTrans, self).build(input_shape)
    
    def call(self, inputs):
        # Get input tensors; in this case it's just one tensor: X = the input to the model
        x = inputs

        # Get the batchsize which is tf.shape(x)[0]
        batchsize = tf.shape(x)[0]

        # Get the shape of the input data
        input_shape = K.int_shape(x)
        
        # Select only 'highway' length of input to create output
        output = x[:,-self.hw:,:]
        
        # Permute axis 1 and 2. axis=2 is the the dimension having different time-series
        # This dimension should be equal to 'm' which is the number of time-series.
        output = tf.transpose(output, perm=[0,2,1])
        
        # Merge axis 0 and 1 in order to change the batch size
        output = tf.reshape(output, [batchsize * input_shape[2], self.hw])
        
        # Adjust the output shape by setting back the batch size dimension to None
        output_shape = tf.TensorShape([None]).concatenate(output.get_shape()[1:])
        
        return output
    
    def compute_output_shape(self, input_shape):
        # Set the shape of axis=1 to be hw since the batchsize is NULL
        shape = tf.TensorShape(input_shape).as_list()
        shape[1] = self.hw
        
        return tf.TensorShape(shape)

    def get_config(self):
        config = {'hw': self.hw}
        base_config = super(PreARTrans, self).get_config()
        
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config):
        return cls(**config)

    
class PostARTrans(tf.keras.layers.Layer):
    def __init__(self, m, **kwargs):
        #
        # m: Number of timeseries
        #
        self.m = m
        super(PostARTrans, self).__init__(**kwargs)
    
    def build(self, input_shape):
        super(PostARTrans, self).build(input_shape)
    
    def call(self, inputs):
        # Get input tensors
	# - First one is the output of the Dense(1) layer which we will operate on
	# - The second is the oiriginal model input tensor which we will use to get
	#   the original batchsize
        x, original_model_input = inputs

        # Get the batchsize which is tf.shape(original_model_input)[0]
        batchsize = tf.shape(original_model_input)[0]

        # Get the shape of the input data
        input_shape = K.int_shape(x)
        
        # Reshape the output to have the batch size equal to the original batchsize before PreARTrans
        # and the second dimension as the number of timeseries
        output = tf.reshape(x, [batchsize, self.m])
        
        # Adjust the output shape by setting back the batch size dimension to None
        output_shape = tf.TensorShape([None]).concatenate(output.get_shape()[1:])
        
        return output
    
    def compute_output_shape(self, input_shape):
        # Adjust shape[1] to be equal 'm'
        shape = tf.TensorShape(input_shape).as_list()
        shape[1] = self.m
        
        return tf.TensorShape(shape)

    def get_config(self):
        config = {'m': self.m}
        base_config = super(PostARTrans, self).get_config()
        
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [ ]:
class DataUtil(object):
    #
    # This class contains data specific information.
    # It does the following:
    #  - Read data from file
    #  - Normalise it
    #  - Split it into train, dev (validation) and test
    #  - Create X and Y for each of the 3 sets (train, dev, test) according to the following:
    #    Every sample (x, y) shall be created as follows:
    #     - x --> window number of values
    #     - y --> one value that is at horizon in the future i.e. that is horizon away past the last value of x
    #    This way X and Y will have the following dimensions:
    #     - X [number of samples, window, number of multivariate time series]
    #     - Y [number of samples, number of multivariate time series]
    
    def __init__(self, filename, train, valid, horizon, window, normalise = 2):
        try:
            fin = open(filename)

            log.debug("Start reading data")
            self.rawdata   = np.loadtxt(fin, delimiter=',')
            log.debug("End reading data")

            self.w         = window
            self.h         = horizon
            self.data      = np.zeros(self.rawdata.shape).reshape(-1,1)
            self.n, self.m = self.data.shape
            self.normalise = normalise
            self.scale     = np.ones(self.m)
        
            self.normalise_data(normalise)
            self.split_data(train, valid)
        except IOError as err:
            # In case file is not found, all of the above attributes will not have been created
            # Hence, in order to check if this call was successful, you can call hasattr on this object 
            # to check if it has attribute 'data' for example
            log.error("Error opening data file ... %s", err)
        
        
    def normalise_data(self, normalise):
        log.debug("Normalise: %d", normalise)

        if normalise == 0: # do not normalise
            self.data = self.rawdata
        
        if normalise == 1: # same normalisation for all timeseries
            self.data = self.rawdata / np.max(self.rawdata)
        
        if normalise == 2: # normalise each timeseries alone. This is the default mode
            for i in range(self.m):
                self.scale[i] = np.max(np.abs(self.rawdata[:, i]))
                self.data[:, i] = self.rawdata[:, i] / self.scale[i]
        
        self.data = self.data.reshape(-1,1)
    def split_data(self, train, valid):
        log.info("Splitting data into training set (%.2f), validation set (%.2f) and testing set (%.2f)", train, valid, 1 - (train + valid))

        train_set = range(self.w + self.h - 1, int(train * self.n))
        valid_set = range(int(train * self.n), int((train + valid) * self.n))
        test_set  = range(int((train + valid) * self.n), self.n)
        
        self.train = self.get_data(train_set)
        self.valid = self.get_data(valid_set)
        self.test  = self.get_data(test_set)
        
    def get_data(self, rng):
        n = len(rng)
        #rng=[23,24,25...]
        X = np.zeros((n, self.w, self.m))
        Y = np.zeros((n, self.m))
        
        for i in range(n-1):
            end   = rng[i] - self.h + 1
            start = end - self.w
            
            X[i,:,:] = self.data[start:end, :]
            Y[i,:]   = self.data[rng[i]+1,:]
        X=X[:-1]
        Y=Y[:-1]
        return [X, Y]

In [ ]:
def rse(y_true, y_pred):
    #
    # The formula is:
    #           K.sqrt(K.sum(K.square(y_true - y_pred)))     
    #    RSE = -----------------------------------------------
    #           K.sqrt(K.sum(K.square(y_true_mean - y_true)))       
    #
    #           K.sqrt(K.sum(K.square(y_true - y_pred))/(N-1))
    #        = ----------------------------------------------------
    #           K.sqrt(K.sum(K.square(y_true_mean - y_true)/(N-1)))
    #
    #
    #           K.sqrt(K.mean(K.square(y_true - y_pred)))
    #        = ------------------------------------------
    #           K.std(y_true)
    #
    num = K.sqrt(K.mean(K.square(y_true - y_pred), axis=None))
    den = K.std(y_true, axis=None)
    
    return num / den


def corr(y_true, y_pred):
    #
    # This function calculates the correlation between the true and the predicted outputs
    #
    num1 = y_true - K.mean(y_true, axis=0)
    num2 = y_pred - K.mean(y_pred, axis=0)
    
    num  = K.mean(num1 * num2, axis=0)
    den  = K.std(y_true, axis=0) * K.std(y_pred, axis=0)
    
    return K.mean(num / den)

#
# A function that compiles 'model' after setting the appropriate:
# - optimiser function passed via init 
# - learning rate passed via init
# - loss function also set in init
# - metrics
#
import datetime
def ModelCompile(model, init):
    # Select the appropriate optimiser and set the learning rate from input values (arguments)
    opt  = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    %load_ext tensorboard
    log_dir = "logs/my_board/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# 텐서보드 콜백 정의 하기
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    # Compile using the previously defined metrics
    model.compile(optimizer = opt, loss = tf.losses.MeanSquaredError(), metrics=[rse])
    tensorboard=tensorboard_callback
    tensorboard = None

    return tensorboard

    

In [ ]:
#print(Data.train[0].shape)
#print(Data2.train[0].shape)
#print(Data3.train[0].shape)

In [ ]:
import pandas as pd
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import numpy as np
%matplotlib inline  
plt.rc('font', family='NanumBarunGothic') 

import seaborn as sns
import random

#Low pass filter
from scipy import signal

#timestamp
import datetime
import time

#import libaray
import tensorflow as tf

In [ ]:
fin = open('/content/drive/MyDrive/energy/LSTNet/data/df_elec.txt')
rawdata   = np.loadtxt(fin, delimiter=',')

In [ ]:
class LSTNetInit(object):

    def __init__(self):
        return
args =()
lstnet_init = LSTNetInit()

In [ ]:
lstnet_init.trainpercent=0.6
lstnet_init.validpercent=0.2
lstnet_init.horizon=0
lstnet_init.window=24
lstnet_init.normalise=0

logger_name = "lstnet"

from __main__ import logger_name
import logging
log = logging.getLogger(logger_name)
Data = DataUtil('/content/drive/MyDrive/energy/LSTNet/data/df_elec.txt',lstnet_init.trainpercent,lstnet_init.validpercent,lstnet_init.horizon,lstnet_init.window,lstnet_init.normalise)

In [ ]:
lstnet_init.normalise=1
Data2 = DataUtil('/content/drive/MyDrive/energy/LSTNet/data/df_temp.txt',lstnet_init.trainpercent,lstnet_init.validpercent,lstnet_init.horizon,lstnet_init.window,lstnet_init.normalise)

In [ ]:
lstnet_init.normalise=1

Data3 = DataUtil('/content/drive/MyDrive/energy/LSTNet/data/df_korea_elec.txt',lstnet_init.trainpercent,lstnet_init.validpercent,lstnet_init.horizon,lstnet_init.window,lstnet_init.normalise)


In [ ]:
def LSTNetModel(init, input_shape):
   # self.init=init
    #self.input_shape=input_shape  
    # m is the number of time-series
    m = input_shape[2]

    # Get tensor shape except batchsize
    tensor_shape = input_shape[1:]
    
    if K.image_data_format() == 'channels_last':
        ch_axis = 3
    else:
        ch_axis = 1
    
    X = Input(shape = tensor_shape)

    # CNN
    if init.CNNFilters > 0 and init.CNNKernel > 0:
        # Add an extra dimension of size 1 which is the channel dimension in Conv2D
        C = Reshape((input_shape[1], input_shape[2], 1))(X)

	# Apply a Conv2D that will transform it into data of dimensions (batchsize, time, 1, NumofFilters)
        C = Conv2D(filters=init.CNNFilters, kernel_size=(init.CNNKernel, m), kernel_initializer=init.initialiser)(C)
        C = Dropout(init.dropout)(C)

        # Adjust data dimensions by removing axis=2 which is always equal to 1
        c_shape = K.int_shape(C)
        C = Reshape((c_shape[1], c_shape[3]))(C)
    else:
	# If configured not to apply CNN, copy the input
        C = X
    
    # GRU
    # Apply a GRU layer (with activation set to 'relu' as per the paper) and take the returned states as result
    _, R = GRU(init.GRUUnits, activation="relu", return_sequences = False, return_state = True)(C)
    R    = Dropout(init.dropout)(R)
    
    # SkipGRU
    if init.skip > 0:
	# Calculate the number of values to use which is equal to the window divided by how many time values to skip
        pt   = int(init.window / init.skip)

        S    = PreSkipTrans(pt, int((init.window - init.CNNKernel + 1) / pt))(C)
        _, S = GRU(init.SkipGRUUnits, activation="relu", return_sequences = False, return_state = True)(S)
        S    = PostSkipTrans(int((init.window - init.CNNKernel + 1) / pt))([S,X])

	# Concatenate the outputs of GRU and SkipGRU
        R    = Concatenate(axis=1)([R,S])
    
    # Dense layer
    Y = Flatten()(R)
    Y = Dense(m)(Y)
    
    # AR
    if init.highway > 0:
        Z = PreARTrans(init.highway)(X)
        Z = Flatten()(Z)
        Z = Dense(1)(Z)
        Z = PostARTrans(m)([Z,X])

	# Generate output as the summation of the Dense layer output and the AR one
        Y = Add()([Y,Z])
    
    # Generate Model
    model1 = Model(inputs = X, outputs = Y)

    ###############################################################################
    X2 = Input(shape = tensor_shape)

    # CNN
    if init.CNNFilters > 0 and init.CNNKernel > 0:
        # Add an extra dimension of size 1 which is the channel dimension in Conv2D
        C = Reshape((input_shape[1], input_shape[2], 1))(X2)

	# Apply a Conv2D that will transform it into data of dimensions (batchsize, time, 1, NumofFilters)
        C = Conv2D(filters=init.CNNFilters, kernel_size=(init.CNNKernel, m), kernel_initializer=init.initialiser)(C)
        C = Dropout(init.dropout)(C)

        # Adjust data dimensions by removing axis=2 which is always equal to 1
        c_shape = K.int_shape(C)
        C = Reshape((c_shape[1], c_shape[3]))(C)
    else:
	# If configured not to apply CNN, copy the input
        C = X2
    
    # GRU
    # Apply a GRU layer (with activation set to 'relu' as per the paper) and take the returned states as result
    _, R = GRU(init.GRUUnits, activation="relu", return_sequences = False, return_state = True)(C)
    R    = Dropout(init.dropout)(R)
    
    # SkipGRU
    if init.skip > 0:
	# Calculate the number of values to use which is equal to the window divided by how many time values to skip
        pt   = int(init.window / init.skip)

        S    = PreSkipTrans(pt, int((init.window - init.CNNKernel + 1) / pt))(C)
        _, S = GRU(init.SkipGRUUnits, activation="relu", return_sequences = False, return_state = True)(S)
        S    = PostSkipTrans(int((init.window - init.CNNKernel + 1) / pt))([S,X2])

	# Concatenate the outputs of GRU and SkipGRU
        R    = Concatenate(axis=1)([R,S])
    
    # Dense layer
    Y2 = Flatten()(R)
    Y2 = Dense(m)(Y2)
    
    # AR
    if init.highway > 0:
        Z = PreARTrans(init.highway)(X2)
        Z = Flatten()(Z)
        Z = Dense(1)(Z)
        Z = PostARTrans(m)([Z,X2])

	# Generate output as the summation of the Dense layer output and the AR one
        Y2 = Add()([Y2,Z])
    
    # Generate Model
    model2 = Model(inputs = X2, outputs = Y2)
    ###########################################################################
    X3 = Input(shape = tensor_shape)

    # CNN
    if init.CNNFilters > 0 and init.CNNKernel > 0:
        # Add an extra dimension of size 1 which is the channel dimension in Conv2D
        C = Reshape((input_shape[1], input_shape[2], 1))(X3)

	# Apply a Conv2D that will transform it into data of dimensions (batchsize, time, 1, NumofFilters)
        C = Conv2D(filters=init.CNNFilters, kernel_size=(init.CNNKernel, m), kernel_initializer=init.initialiser)(C)
        C = Dropout(init.dropout)(C)

        # Adjust data dimensions by removing axis=2 which is always equal to 1
        c_shape = K.int_shape(C)
        C = Reshape((c_shape[1], c_shape[3]))(C)
    else:
	# If configured not to apply CNN, copy the input
        C = X3
    
    # GRU
    # Apply a GRU layer (with activation set to 'relu' as per the paper) and take the returned states as result
    _, R = GRU(init.GRUUnits, activation="relu", return_sequences = False, return_state = True)(C)
    R    = Dropout(init.dropout)(R)
    
    # SkipGRU
    if init.skip > 0:
	# Calculate the number of values to use which is equal to the window divided by how many time values to skip
        pt   = int(init.window / init.skip)

        S    = PreSkipTrans(pt, int((init.window - init.CNNKernel + 1) / pt))(C)
        _, S = GRU(init.SkipGRUUnits, activation="relu", return_sequences = False, return_state = True)(S)
        S    = PostSkipTrans(int((init.window - init.CNNKernel + 1) / pt))([S,X3])

	# Concatenate the outputs of GRU and SkipGRU
        R    = Concatenate(axis=1)([R,S])
    
    # Dense layer
    Y3 = Flatten()(R)
    Y3 = Dense(m)(Y3)
    
    # AR
    if init.highway > 0:
        Z = PreARTrans(init.highway)(X3)
        Z = Flatten()(Z)
        Z = Dense(1)(Z)
        Z = PostARTrans(m)([Z,X3])

	# Generate output as the summation of the Dense layer output and the AR one
        Y3 = Add()([Y3,Z])
    
    # Generate Model
    model3 = Model(inputs = X3, outputs = Y3)
    combined = keras.layers.concatenate([model1.output, model2.output, model3.output])
    z = Dense(1, activation="linear")(combined)
    model = Model(inputs=[X, X2,X3], outputs=z) 
    opt  = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer = opt, loss = tf.losses.MeanSquaredError(), metrics=[rse])
    return model

In [ ]:
lstnet_init.CNNFilters=60
lstnet_init.CNNKernel=3
lstnet_init.initialiser='glorot_uniform'
lstnet_init.dropout=0.1
lstnet_init.GRUUnits=120
lstnet_init.skip=24
lstnet_init.highway=24
lstnet_init.SkipGRUUnits=0
from tensorflow import keras
#tuner=kt.RandomSearch(LSTNetModel(lstnet_init, Data.train[0].shape), objective='val_loss',max_trials=5)
lstnet=LSTNetModel(lstnet_init, Data.train[0].shape)
lstnet_tensorboard = ModelCompile(lstnet, lstnet_init)
#lstnet=LSTNetModel(lstnet_init, Data.train[0].shape)


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
Data.train[1]

array([[4990.83854479],
       [4816.23851571],
       [4675.08638126],
       ...,
       [5025.75394864],
       [4877.68875127],
       [4740.05891203]])

In [ ]:
X=[Data.train[0],Data2.train[0],Data3.train[0]]
Y=[Data.train[1]]

val_x=[Data.valid[0],Data2.valid[0],Data3.valid[0]]
val_y=[Data.valid[1]]



In [ ]:
history = lstnet.fit(
                x = X,
                y = Y,
                epochs = 400,
                batch_size = 64,
                validation_data = (val_x, val_y),
        )

Epoch 1/400
76/76 [==============================] - 13s 76ms/step - loss: 3083740.7500 - rse: 1.6581 - val_loss: 160690.5469 - val_rse: 1.0921
Epoch 2/400
76/76 [==============================] - 5s 62ms/step - loss: 407008.1250 - rse: 0.7498 - val_loss: 145322.1250 - val_rse: 1.0685
Epoch 3/400
76/76 [==============================] - 5s 61ms/step - loss: 356477.3438 - rse: 0.6990 - val_loss: 128078.9531 - val_rse: 1.1643
Epoch 4/400
76/76 [==============================] - 5s 62ms/step - loss: 302728.5938 - rse: 0.6448 - val_loss: 106265.1406 - val_rse: 1.1011
Epoch 5/400
76/76 [==============================] - 5s 61ms/step - loss: 275981.0625 - rse: 0.6190 - val_loss: 109643.7266 - val_rse: 1.2888
Epoch 6/400
76/76 [==============================] - 5s 61ms/step - loss: 246400.9062 - rse: 0.5849 - val_loss: 94363.7266 - val_rse: 1.2183
Epoch 7/400
76/76 [==============================] - 5s 62ms/step - loss: 213666.5156 - rse: 0.5431 - val_loss: 68892.7188 - val_rse: 0.9022
Epoch 